In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import time

In [2]:
start_time = time.time()


df = pd.read_parquet('../time_seires/year/dfTime_TLC-2022.parquet', engine='pyarrow')
df.head(10)


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

執行時間: 0.41016674041748047 秒


In [3]:
df.shape

(5100018, 9)

In [4]:
uber = df[(df['Name'] == 'uber') & (df['PULocationID'] == 3)].reset_index(drop=True)
uber.head()

,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
0,uber,2022,1,1,0,5,3,True,39
1,uber,2022,1,1,1,5,3,True,44
2,uber,2022,1,1,2,5,3,True,38
3,uber,2022,1,1,3,5,3,True,31
4,uber,2022,1,1,4,5,3,True,24


In [5]:
uber.shape

(8758, 9)

In [6]:
uber.tail(10)

,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
8748,uber,2022,12,31,14,5,3,True,56
8749,uber,2022,12,31,15,5,3,True,60
8750,uber,2022,12,31,16,5,3,True,60
8751,uber,2022,12,31,17,5,3,True,51
8752,uber,2022,12,31,18,5,3,True,56
8753,uber,2022,12,31,19,5,3,True,77
8754,uber,2022,12,31,20,5,3,True,63
8755,uber,2022,12,31,21,5,3,True,53
8756,uber,2022,12,31,22,5,3,True,58
8757,uber,2022,12,31,23,5,3,True,50


In [7]:
uber['date'] = uber['month'].astype('str') + '/' + uber['day'].astype('str')
uber['date']

# import plotly.express as px
# fig = px.line(uber, y='次數')
# fig.show()

0         1/1
1         1/1
2         1/1
3         1/1
4         1/1
        ...  
8753    12/31
8754    12/31
8755    12/31
8756    12/31
8757    12/31
Name: date, Length: 8758, dtype: object

# Keras LSTM

In [8]:
# from sklearn.preprocessing import MinMaxScaler

# training_set = uber['次數'].values.reshape(-1, 1)


# scaler = MinMaxScaler()
# training_set_scaled = scaler.fit_transform(training_set)

In [9]:
# X_train = []   #預測點的前 60 天的資料
# y_train = []   #預測點
# for i in range(48, 7000):  # 1258 是訓練集總數
#     X_train.append(training_set_scaled[i-48:i, 0])
#     y_train.append(training_set_scaled[i, 0])
# X_train, y_train = np.array(X_train), np.array(y_train)

In [10]:
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [11]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras.layers import Dropout

# regressor = Sequential()

In [12]:
# regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units=50, return_sequences=True))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units=50, return_sequences=True))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units=50))
# regressor.add(Dropout(0.2))

# regressor.add(Dense(units=1))

In [13]:
# regressor.compile(optimizer='adam', loss='mean_squared_error')

# 進行訓練
# history = regressor.fit(X_train, y_train, epochs=50, batch_size=32)

## scikit-learn

In [14]:
df.head()

,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
0,lyft,2022,1,1,0,5,3,True,16
1,lyft,2022,1,1,0,5,4,True,49
2,lyft,2022,1,1,0,5,5,True,1
3,lyft,2022,1,1,0,5,6,True,1
4,lyft,2022,1,1,0,5,7,True,115


In [15]:
df.shape

(5100018, 9)

In [16]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
df['Name'] = label.fit_transform(df['Name'])

df['Name'].value_counts()

1    2177568
0    2115281
2     807169
Name: Name, dtype: int64

In [17]:
df = df.drop('year', axis=1)
df = df.astype({'is_holiday': 'int'})
df = df.rename(columns={'次數': 'count'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100018 entries, 0 to 5100017
Data columns (total 8 columns):
 #   Column        Dtype
---  ------        -----
 0   Name          int32
 1   month         int32
 2   day           int32
 3   hour          int32
 4   weekday       int32
 5   PULocationID  int64
 6   is_holiday    int32
 7   count         int64
dtypes: int32(6), int64(2)
memory usage: 194.6 MB


In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor



X = df.drop('count', axis=1)
y = df['count']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


# models = {'KNN': KNeighborsRegressor(),
#           'SVR': SVR(),
#           'Decision Tree': DecisionTreeRegressor(), 
#           'RandomForest': RandomForestRegressor(),
#           'XGBoost': XGBRegressor(n_estimators=500, 
#                                   learning_rate=0.05, 
#                                   n_jobs=-1, 
#                                   random_state=0),
#           'LightGBM': LGBMRegressor(n_estimators=500, 
#                                     learning_rate=0.1, 
#                                     n_jobs=-1, 
#                                     random_state=0)}
# results = []

# for model in models.values():
#     kf = KFold(n_splits=6, random_state=42, shuffle=True)
#     cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kf)
#     results.append(cv_results)

# # plt.boxplot(results, labels=models.keys())
# sns.boxplot(results, x=models.keys())
# plt.show()

In [19]:
# model = RandomForestRegressor(random_state=1, n_jobs=-1)
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# print(f'MAE: {mae}\nR2: {r2}')

## LightGBM

In [ ]:
start_time = time.time()


from sklearn.model_selection import GridSearchCV
kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {'n_estimators': range(100, 501, 100),
              'learning_rate': np.logspace(0, -2, 3), 
              'max_depth': range(3, 20, 2)}

lgbm = LGBMRegressor(random_state=0)

model_cv = GridSearchCV(lgbm, param_grid, cv=kf)
model_cv.fit(X_train, y_train)
print(model_cv.best_params_, model_cv.best_score_)


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

C:\Users\Ticy\anaconda3\envs\fhv\lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Ticy\anaconda3\envs\fhv\lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025032 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 324
[LightGBM] [Info] Number of data points in the train set: 3264011, number of used features: 7
[LightGBM] [Info] Start training from score 49.191474


In [ ]:
# start_time = time.time()


# model = LGBMRegressor(n_estimators=1000, 
#                       learning_rate=0.1, 
#                       subsample=0.6
#                       random_state=0)
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'MAE: {mae}\nR2: {r2}')


# end_time = time.time()
# execution_time = end_time - start_time
# print("執行時間:", execution_time, "秒")

## XGBoost

In [ ]:
start_time = time.time()


from sklearn.model_selection import GridSearchCV
kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {'n_estimators': range(100, 501, 100),
              'learning_rate': np.logspace(0,-2, 3)}

xgbr = XGBRegressor(random_state=0)

model_cv = GridSearchCV(xgbr, param_grid, cv=kf)
model_cv.fit(X_train, y_train)

# 查看表現最好的超參數和平均交叉驗證分數
print(model_cv.best_params_, model_cv.best_score_)


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

In [ ]:
# model = XGBRegressor(n_estimators=1000, learning_rate=0.1, n_jobs=-1, random_state=0)
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'MAE: {mae}\nR2: {r2}')

In [ ]:
from lazypredict.Supervised import LazyRegressor

LC = LazyRegressor(ignore_warnings=True, random_state=0, verbose=False)
models, predictions = LC.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/42 [00:00<?, ?it/s]

## NN

In [ ]:
# start_time = time.time()


# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Input
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.layers import LeakyReLU

# n_cols = X_train.shape[1]

# model = Sequential([Dense(14, input_shape=(n_cols,)),
#                     BatchNormalization(),
#                     LeakyReLU(),
#                     Dense(14, input_shape=(n_cols,)),
#                     BatchNormalization(),
#                     LeakyReLU(),
#                     Dense(1)
# ])

# model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])
# model.summary()

# # 擬合X, y
# history = model.fit(X_train, y_train, validation_split=0.3, batch_size=100, epochs=10, verbose=2)


# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.legend(['training', 'validation'], loc='upper left')
# plt.show()


# result = model.evaluate(X_test, y_test)
# print(result)


# end_time = time.time()
# execution_time = end_time - start_time
# print("執行時間:", execution_time, "秒")